In [3]:
import pandas as pd
import pdal
import json
import geopandas as gpd
import numpy as np

In [4]:
gdf_articulacao = gpd.read_file("zip://data/SIRGAS_SHP_quadriculamdt.zip!/SIRGAS_SHP_quadriculamdt/")

In [5]:
gdf_articulacao.set_crs(epsg=31983, inplace=True)

,qmdt_cod,geometry
0,3212-353,"POLYGON ((329132.004 7367829.458, 329125.190 7..."
1,3212-344,"POLYGON ((328601.255 7367823.210, 328594.420 7..."
2,3212-343,"POLYGON ((328070.502 7367816.960, 328063.648 7..."
3,3211-464,"POLYGON ((327539.756 7367810.622, 327532.876 7..."
4,3211-463,"POLYGON ((327008.999 7367804.285, 327002.102 7..."
...,...,...
5357,3234-241,"POLYGON ((331960.582 7352864.137, 331967.325 7..."
5358,3234-224,"POLYGON ((333544.473 7353459.402, 333551.152 7..."
5359,3336-444,"POLYGON ((331725.429 7373051.662, 332256.363 7..."
5360,3336-442,"POLYGON ((331718.736 7373628.441, 332249.692 7..."


In [6]:
gdf_articulacao.set_index('qmdt_cod').loc['3315-361']

geometry    POLYGON ((323593.445 7386800.007, 323586.469 7...
Name: 3315-361, dtype: geometry

In [7]:
#gdf_articulacao.set_index('qmdt_cod').iloc[3315-361].geometry.exterior.coords
coords = [[xy[0], xy[1]] for xy in gdf_articulacao.set_index('qmdt_cod').loc['3315-361'].geometry.exterior.coords]
xy_max = np.max(np.array(coords), axis=0) 
xy_min = np.min(np.array(coords), axis=0)

In [8]:
np.array(coords)

array([[ 323593.445, 7386800.007],
       [ 323586.469, 7387376.793],
       [ 324117.945, 7387383.21 ],
       [ 324124.9  , 7386806.425],
       [ 323593.445, 7386800.007]])

In [9]:
np.ceil(xy_max * 2) - np.ceil(xy_min * 2)

array([1077., 1166.])

In [10]:
xy_min, xy_max

(array([ 323586.469, 7386800.007]), array([ 324124.9 , 7387383.21]))

In [11]:
np.floor(xy_min * 2)/2

array([ 323586., 7386800.])

In [20]:
## GERA MDS e MDT

laz = [
    {
        "type":"readers.las",
        "filename":"sample-data/MDS_3315-361_1000.laz"
    },
    {
        "filename":"sample-result/MDS-3315-361.tiff",
        "gdaldriver":"GTiff",
        "output_type":"max",
        "resolution":"0.5",
        "type": "writers.gdal",
        "gdalopts":"COMPRESS=ZSTD, PREDICTOR=3, BIGTIFF=YES",
    },
    {
        "type": "filters.delaunay",
        "where": "(Classification == 2)"
    },
    {
        "type": "filters.faceraster",
        "resolution": 0.5,
        "width": 1077,
        "height": 1166,
        "origin_x": 323586,
        "origin_y": 7386800
    },
    {
        "filename":"sample-result/MDT-3315-361.tiff",
        "type": "writers.raster",
        # "filename":f"results/DTM-{grid_number}.tiff",
        "gdaldriver":"GTiff",
        "data_type": "float32",
        "gdalopts":"COMPRESS=ZSTD, PREDICTOR=3, BIGTIFF=YES",
    }, 
]

In [21]:
pipeline = pdal.Pipeline(json.dumps(laz))
n_points = pipeline.execute()
print(f'Pipeline selected {n_points} points')

Pipeline selected 5365968 points
